In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [37]:
data = pd.read_csv('data2.csv', delimiter=',')
data.head()

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,4,0,12/7/2009,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   RefId                              72983 non-null  int64  
 1   IsBadBuy                           72983 non-null  int64  
 2   PurchDate                          72983 non-null  object 
 3   Auction                            72983 non-null  object 
 4   VehYear                            72983 non-null  int64  
 5   VehicleAge                         72983 non-null  int64  
 6   Make                               72983 non-null  object 
 7   Model                              72983 non-null  object 
 8   Trim                               70623 non-null  object 
 9   SubModel                           72975 non-null  object 
 10  Color                              72975 non-null  object 
 11  Transmission                       72974 non-null  obj

In [39]:
data['WarrantyCost'].value_counts(dropna=False)

920     2870
1974    2407
2152    2082
1389    2078
1215    2006
        ... 
1872       1
6519       1
1361       1
4241       1
1931       1
Name: WarrantyCost, Length: 281, dtype: int64

Я решила убрать следующие столбцы: 

1) VehYear и PurchDate, так как есть VehicleAge

2) Model, Trim, SubModel, Color так как при их кодировании пораждается слишком много столбцов

3) WheelType, так как есть WheelTypeID

4) PRIMEUNIT, AUCGUART, так как слишком много нанов

5) VNZIP1, VNST так как считаю что место покупки никак не влияет на машину

In [40]:
data.drop(['RefId', 'VehYear', 'Model', 'Trim', 'SubModel', 'Color', 'WheelType', 'PRIMEUNIT', 'AUCGUART', 'VNZIP1', 'VNST', 'PurchDate'], axis=1, inplace=True)

In [41]:
data.head()

,IsBadBuy,Auction,VehicleAge,Make,Transmission,WheelTypeID,VehOdo,Nationality,Size,TopThreeAmericanName,...,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,BYRNO,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,0,ADESA,3,MAZDA,AUTO,1.0,89046,OTHER ASIAN,MEDIUM,OTHER,...,13600.0,7451.0,8552.0,11597.0,12409.0,21973,FL,7100.0,0,1113
1,0,ADESA,5,DODGE,AUTO,1.0,93593,AMERICAN,LARGE TRUCK,CHRYSLER,...,12572.0,7456.0,9222.0,11374.0,12791.0,19638,FL,7600.0,0,1053
2,0,ADESA,4,DODGE,AUTO,2.0,73807,AMERICAN,MEDIUM,CHRYSLER,...,8457.0,4035.0,5557.0,7146.0,8702.0,19638,FL,4900.0,0,1389
3,0,ADESA,5,DODGE,AUTO,1.0,65617,AMERICAN,COMPACT,CHRYSLER,...,5690.0,1844.0,2646.0,4375.0,5518.0,19638,FL,4100.0,0,630
4,0,ADESA,4,FORD,MANUAL,2.0,69367,AMERICAN,COMPACT,FORD,...,8707.0,3247.0,4384.0,6739.0,7911.0,19638,FL,4000.0,0,1020


Я решила сделать отдельный столбец средних значений столбцов MMR...

In [42]:
MMR = ['MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice',
                        'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice',
                        'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice',
                        'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice']
                        
    
data['MMRAve'] = sum(data[ave] for ave in MMR) /len(MMR)
data = data.drop(MMR, axis=1)

In [43]:
data.head()

,IsBadBuy,Auction,VehicleAge,Make,Transmission,WheelTypeID,VehOdo,Nationality,Size,TopThreeAmericanName,BYRNO,VNST,VehBCost,IsOnlineSale,WarrantyCost,AuctionAve
0,0,ADESA,3,MAZDA,AUTO,1.0,89046,OTHER ASIAN,MEDIUM,OTHER,21973,FL,7100.0,0,1113,10403.625
1,0,ADESA,5,DODGE,AUTO,1.0,93593,AMERICAN,LARGE TRUCK,CHRYSLER,19638,FL,7600.0,0,1053,9943.625
2,0,ADESA,4,DODGE,AUTO,2.0,73807,AMERICAN,MEDIUM,CHRYSLER,19638,FL,4900.0,0,1389,6100.250
3,0,ADESA,5,DODGE,AUTO,1.0,65617,AMERICAN,COMPACT,CHRYSLER,19638,FL,4100.0,0,630,3662.375
4,0,ADESA,4,FORD,MANUAL,2.0,69367,AMERICAN,COMPACT,FORD,19638,FL,4000.0,0,1020,5959.750


Все nan заполняю медианами или модой.

In [45]:
for col in ['VehicleAge', 'VehOdo', 'VehBCost', 'WarrantyCost', 'AuctionAve']:
    data[col] = data[col].fillna(data[col].median())
    
for col in ['Auction', 'Make', 'Transmission', 'WheelTypeID', 'Nationality', 'Size', 'TopThreeAmericanName', 'BYRNO', 'IsOnlineSale']:
    mode = data[col].mode()[0]
    data[col] = data[col].fillna(mode)

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   IsBadBuy              72983 non-null  int64  
 1   Auction               72983 non-null  object 
 2   VehicleAge            72983 non-null  int64  
 3   Make                  72983 non-null  object 
 4   Transmission          72983 non-null  object 
 5   WheelTypeID           72983 non-null  float64
 6   VehOdo                72983 non-null  int64  
 7   Nationality           72983 non-null  object 
 8   Size                  72983 non-null  object 
 9   TopThreeAmericanName  72983 non-null  object 
 10  BYRNO                 72983 non-null  int64  
 11  VNST                  72983 non-null  object 
 12  VehBCost              72983 non-null  float64
 13  IsOnlineSale          72983 non-null  int64  
 14  WarrantyCost          72983 non-null  int64  
 15  AuctionAve         

In [47]:
data = pd.get_dummies(data, columns = ['Auction', 'Make', 'Transmission', 'WheelTypeID', 'Nationality', 'Size', 'TopThreeAmericanName', 'BYRNO', 'IsOnlineSale'])

In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Columns: 182 entries, IsBadBuy to IsOnlineSale_1
dtypes: float64(2), int64(4), uint8(176)
memory usage: 15.6 MB


In [49]:
X = data.copy()
X.drop('IsBadBuy', axis=1, inplace=True)
y = data['IsBadBuy']

In [18]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Columns: 181 entries, VehicleAge to IsOnlineSale_1
dtypes: float64(2), int64(3), uint8(176)
memory usage: 15.0 MB


In [19]:
pca = PCA(n_components=7)
pca.fit(X)
X = pca.transform(X)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
model = RandomForestClassifier(max_depth=6, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)

In [69]:
y_pred

array([[0.82637167, 0.17362833],
       [0.86536761, 0.13463239],
       [0.87683346, 0.12316654],
       ...,
       [0.89983821, 0.10016179],
       [0.92494653, 0.07505347],
       [0.91926893, 0.08073107]])

In [80]:
predictionClass = np.array([1 if y_pred[i][1] > 0.15 else 0 for i in range(y_pred.shape[0])])
print(classification_report(y_test, predictionClass))

              precision    recall  f1-score   support

           0       0.91      0.79      0.85     12850
           1       0.23      0.45      0.30      1747

    accuracy                           0.75     14597
   macro avg       0.57      0.62      0.57     14597
weighted avg       0.83      0.75      0.78     14597



In [82]:
print(roc_auc_score(y_test, predictionClass))

0.6208653856861902
